In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import scipy.stats as stats

In [2]:
sym_fea = pd.read_csv("../../../../extract_pubmed/2020_TF-IDF_dropdup.csv",index_col=0)
sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
ori_df = pd.read_csv("CTD_chemicals_diseases.csv",sep=",",skiprows=27)
print(ori_df.shape)
ori_df[:2]

(7682773, 10)


,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
0,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198,therapeutic,NaN,NaN,NaN,4519131
1,10074-G5,C534883,NaN,Adenocarcinoma,MESH:D000230,NaN,MYC,4.08,NaN,26432044


#### extract items  are which therapeutic ，marker/mechanism ，InferenceScore>median
    -3893522

In [5]:
ori_df["DirectEvidence"].value_counts()

marker/mechanism    65958
therapeutic         36969
Name: DirectEvidence, dtype: int64

In [48]:
print(np.mean(ori_df["InferenceScore"]))
#median
inferred_score_median = np.median(ori_df[pd.notnull(ori_df["InferenceScore"])]["InferenceScore"])
print(inferred_score_median)

22.68494197111658
8.4


In [47]:
filtered_df = ori_df[(ori_df["DirectEvidence"]=="therapeutic") | 
                     (ori_df["DirectEvidence"]=="marker/mechanism")|
                     ((pd.notnull(ori_df["InferenceScore"])) & (ori_df["InferenceScore"]>=inferred_score_median))]

print(filtered_df.shape)
filtered_df[:3]

(3893522, 10)


,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
0,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198,therapeutic,NaN,NaN,NaN,4519131
89,"10,10-bis(4-pyridinylmethyl)-9(10H)-anthracenone",C112297,NaN,Epilepsy,MESH:D004827,NaN,FOS,9.06,NaN,15973680
90,"10,10-bis(4-pyridinylmethyl)-9(10H)-anthracenone",C112297,NaN,Epilepsy,MESH:D004827,NaN,KCNQ2,9.06,NaN,16464983|29942082


In [49]:
disdrug_df = filtered_df[["ChemicalID","DiseaseName","DiseaseID"]]
print(disdrug_df.shape)
disdrug_df[:2]

(3893522, 3)


,ChemicalID,DiseaseName,DiseaseID
0,C046983,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198
89,C112297,Epilepsy,MESH:D004827


### Extract common diseases, and related drug sets
- 2692 common diseases
- drug sets：dis_related_drugs

In [50]:
con_dis = list(set(sym_fea.index) & set(disdrug_df["DiseaseName"]))
print(len(con_dis))
con_dis[:10]

2692


['Superinfection',
 'Coloboma',
 'Bronchial Hyperreactivity',
 'Abnormalities, Radiation-Induced',
 'Amyloidosis, Familial',
 'Puberty, Delayed',
 'Pilonidal Sinus',
 'Hematoma, Subdural, Chronic',
 'Cardiovirus Infections',
 'Diffuse Axonal Injury']

In [51]:
con_disdrug_df = disdrug_df[[(i in con_dis) for i in list(disdrug_df["DiseaseName"])]]
con_disdrug_df.head()

,ChemicalID,DiseaseName,DiseaseID
0,C046983,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198
89,C112297,Epilepsy,MESH:D004827
90,C112297,Epilepsy,MESH:D004827
162,C039775,Epilepsy,MESH:D004827
220,C094849,Acute Lung Injury,MESH:D055371


In [52]:
con_disdrug_df.shape

(3365055, 3)

In [53]:
dis_related_drugs = {} #key：disease，value：related drug lists

groupByDis = con_disdrug_df.groupby("DiseaseName")
for key,value in groupByDis:
    dis_related_drugs[key] = list(value["ChemicalID"])
    
print(len(dis_related_drugs))

2692


### caculate Jaccard similarity

In [54]:
predict_score = pd.read_csv("../../visualization/model_predictAll_result.csv",index_col=0)

#extract index
dis_index_inSym = []
sym_fea_index_lower = [s.lower() for s in sym_fea.index]

for dis in con_dis:
    index_inSym = sym_fea_index_lower.index(dis.lower())
    dis_index_inSym.append(index_inSym)
    
print(len(dis_index_inSym))
dis_index_inSym[:5]

E:\Anaconda\envs\tf2\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


2692


[1, 2, 4, 3, 5]

#### Jaccard
- 2692 diseases

In [55]:
scores = []
CTD_confs = [] 

for indexA in tqdm(range(len(con_dis))):
    disA = con_dis[indexA]
    disA_index = dis_index_inSym[list(con_dis).index(disA)]
    
    for indexB in range(indexA+1,len(con_dis)):
        disB = con_dis[indexB]
        disB_index = dis_index_inSym[list(con_dis).index(disB)]
        
        scoreAB = predict_score.loc[disA_index*4268+disB_index]
        scoreBA = predict_score.loc[disB_index*4268+disA_index]
        scores.append((scoreAB+scoreBA)/2)
     
        disA_drugs = dis_related_drugs[disA]
        disB_drugs = dis_related_drugs[disB]
        
        share_drugs_num = len(list(set(disA_drugs) & (set(disB_drugs))))
#         share_drugs_rate = share_drugs_num/(np.log(len(disA_drugs))*np.log(len(disB_drugs)))

        union_drugs_num = len(list(set(disA_drugs) | (set(disB_drugs))))
        share_drugs_jaccard = share_drugs_num/union_drugs_num
    
#         CTD_confs.append(share_drugs_rate)
        CTD_confs.append(share_drugs_jaccard)
        
print(len(CTD_confs))
print(CTD_confs[:5])

100%|██████████████████████████████████████████████████████████████████████████████| 2692/2692 [41:52<00:00,  1.07it/s]


3622086
[0.0, 0.0, 0.0, 0.0, 0.0]


In [56]:
print(len(CTD_confs)) # 3622086 == 2692*2691/2
print(CTD_confs[:5])

print(CTD_confs[100:200])
CTD_confs.count(0.0)

3622086
[0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


2561571

In [58]:
print(CTD_confs[100:200])
CTD_confs.count(0.0)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


2561571

#### Calculate distribution difference

In [59]:
CTD_sim = []
CTD_unsim = []

for i in range(len(scores)):
    score = list(scores[i])[0]
    if score>=0.5:
        CTD_sim.append(CTD_confs[i])
    elif score<0.5:
        CTD_unsim.append(CTD_confs[i])
        
print(len(CTD_sim),"--",len(CTD_unsim)) #806693 -- 2810012

806693 -- 2810012


In [60]:
distribution_dif = stats.mannwhitneyu(CTD_sim,CTD_unsim,alternative='two-sided')
distribution_dif

MannwhitneyuResult(statistic=1217159994523.0, pvalue=0.0)

In [67]:
print(np.mean(CTD_sim))
print(np.mean(CTD_unsim))
print("-----")
print(np.median(CTD_sim))
print(np.median(CTD_unsim))
print("-----")
print(np.quantile(CTD_sim,0.75))
print(np.quantile(CTD_unsim,0.75))


0.011749574878032349
0.00719881088491746
-----
0.0
0.0
-----
0.0036429872495446266
0.0010638297872340426


'相似组的共享药物Jaccard相似性大，√'

In [64]:
# pd.Series(CTD_sim).to_csv("fiteredCTD_sim.csv",index="Jaccard")
# pd.Series(CTD_unsim).to_csv("fiteredCTD_unsim.csv",index="Jaccard")